# Prepare Australian higher education data for analysis
## Overview
I purchased historical higher education data from the Australian Department of Education and Training (DET). This data was provided to me as a set of .csv files that I would need to join and mutate in order to perform meaningful analysis.
Key time-bound features of interest are student to staff ratio, sessional staff proportion, student success rates, student attrition rates, proportion of students that are international students and postgraduate students, overall student counts, and measures of provider specialisation in particular areas of study (measured by engineered features ‘entropy’ and ‘gini impurity’). Key static features relate to provider type, affiliation, ownership and profit status.

In [1]:
import pandas as pd
import numpy as np

## Load data
### Load static provider data

In [2]:
def static():
    '''a function to load static higher education provider data'''
    
    df = pd.read_csv("Provider.csv").fillna(
        value={'affiliation': 'NUHEP', 'affil_status': 'current'})
    
    return df

static().head()

### Load equivalent full-time student load (EFTSL) time series data

In [3]:
def EFTSL():
    '''a function to load equivalent full-time student load (eftsl)'''
    
    df = pd.read_csv('19_381_Onshore_EFTSL.csv').round(decimals=2)
    
    return df

EFTSL().head()

### Load student success and attrition time series data (domestic, international and overall) and join

In [4]:
def load_performance():
    
    '''a function to load, clean and join student success rate and attrition rate data'''
    
    #load success rate data
    success_all = pd.read_csv("19_381_Onshore_success_rate_overall.csv").rename(
        columns = {"success_rate": "success_all"}).drop(['provider_name'], axis=1)
    
    success_dom = pd.read_csv("19_381_Onshore_success_domestic.csv").rename(
        columns = {"success_rate": "success_dom"})[['code_year', 'success_dom']]
    
    success_int = pd.read_csv("19_381_Onshore_success_international.csv").rename(
        columns = {"success_rate": "success_int"})[['code_year', 'success_int']]
    
    # load attrition rate data
    attrition_all = pd.read_csv("19_381_Onshore_attrition_rate_overall.csv").rename(
        columns = {'attrition_rate': 'attrition_all'})[['code_year', 'attrition_all']]
    
    attrition_dom = pd.read_csv("19_381_Onshore_attrition_domestic.csv").rename(
        columns = {'attrition_rate': 'attrition_dom'})[['code_year', 'attrition_dom']]
    
    attrition_int = pd.read_csv("19_381_Onshore_attrition_international.csv").rename(
        columns = {'attrition_rate': 'attrition_int'})[['code_year', 'attrition_int']]
    
    # merge data frames
    df = success_all.merge(success_dom, on='code_year', how = 'outer').merge(
        success_int, on='code_year', how = 'outer').merge(
        attrition_all, on='code_year', how = 'outer').merge(
        attrition_dom, on='code_year', how = 'outer').merge(
        attrition_int, on='code_year', how = 'outer')
    
    return df[df['ref_year'] > 2011]

load_performance().head()

### Load staff-related time series data

In [5]:
eftsl = EFTSL()

def load_staff():

    '''a function to load staff data and create important staff-related features'''
    
    # load .csv
    staff = pd.read_csv('19_381_Academic_staff.csv').rename(
        columns = {'tab1_academic_FTE': 'all_fte', 
                   'tab2_academic_FTE': 'salaried_fte', 
                   'tab3_senior_FTE': 'senior_fte', 
                   'tab3_senior_headcount_mod': 'senior_headcount'})
    
    # calculate pure staffing features
    staff['sessional_fte'] = staff['all_fte'] - staff['salaried_fte']
    staff['sessional_prop'] = staff['sessional_fte'] / staff['all_fte']
    staff['senior_prop'] = staff['senior_fte'] / staff['all_fte']
    
    # join with eftsl data to enable calculation of student:staff ratio
    eftsl_total = eftsl[['code_year', 'EFTSL']].groupby(['code_year']).sum().reset_index()
    staff = staff.merge(eftsl_total, on='code_year').rename(columns={'EFTSL': 'eftsl'})
    
    # calculate student:staff ratios
    staff['ssr_all'] = staff['eftsl'] / staff['all_fte']
    staff['ssr_salaried'] = staff['eftsl'] / staff['salaried_fte']
    
    # select staffing features
    df = staff[['code_year', 
                'all_fte', 
                'salaried_fte', 
                'senior_fte', 
                'senior_headcount', 
                'sessional_fte', 
                'sessional_prop', 
                'senior_prop', 
                'ssr_all',
                'ssr_salaried']]

    return df

load_staff().head()

,code_year,all_fte,salaried_fte,senior_fte,senior_headcount,sessional_fte,sessional_prop,senior_prop,ssr_all,ssr_salaried
0,1019-2012,847.92,720.85,207.33,228.0,127.07,0.149861,0.244516,13.321210,15.669446
1,1034-2012,708.97,527.97,141.78,149.0,181.00,0.255300,0.199980,14.232337,19.111503
2,1055-2012,1630.16,1515.52,480.17,523.0,114.64,0.070324,0.294554,10.995056,11.826766
3,1058-2012,1232.55,965.79,281.19,306.0,266.76,0.216429,0.228137,13.159864,16.794738
4,2154-2012,365.54,254.30,44.20,47.0,111.24,0.304317,0.120917,22.406029,32.207236


### Mutate EFTSL data to produce postgraduate proportions

In [6]:
def build_postgrad():
    
    ''' a function to extract postraguate student propotions from eftsl data'''
    
    # aggregate eftsl at course level by provider by year
    df = eftsl.drop(['provider_code', 'ref_year'], axis=1).groupby(['Course_level','code_year']).sum().reset_index()
    
    # pivot in order to make columnwise calculation
    df = df.pivot(index='code_year', columns='Course_level', values='EFTSL').fillna(value=0)
    
    # flatten index after pivoting
    df.columns = [''.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace = True)
    
    # calculate postgraduate eftsl proportion and drop unnecessary columns
    df['postgrad_prop'] = df['Postgrad by course'] / (df['Postgrad by course'] + df['Undergrad'])
    df = df.drop(['Postgrad by course', 'Undergrad'], axis=1).fillna(value=0)
    
    return df

build_postgrad().head()

,code_year,postgrad_prop
0,1019-2012,0.127903
1,1019-2013,0.151332
2,1019-2014,0.168537
3,1019-2015,0.155800
4,1019-2016,0.142656


### Mutate EFTSL data to produce international student proportions

In [7]:
def build_international():
    
    ''' a function to extract international student proportions from eftsl data'''
    
    # aggregate eftsl at citizenship type (domestic or international) by provider by year
    df = eftsl.drop(['provider_code', 'ref_year'], axis=1).groupby(['citizenship','code_year']).sum().reset_index()
    
    # pivot in order to make columnwise calculation
    df = df.pivot(index='code_year', columns='citizenship', values='EFTSL').fillna(value=0)
    
    # flatten index after pivoting
    df.columns = [''.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace = True)
    
    # calculate international eftsl proportion and drop unnecessary columns
    df['int_prop'] = df['International'] / (df['International'] + df['Domestic'])
    df = df.drop(['International', 'Domestic'], axis=1).fillna(value=0)
    
    return df

build_international().head()

,code_year,int_prop
0,1019-2012,0.137527
1,1019-2013,0.161694
2,1019-2014,0.180512
3,1019-2015,0.168304
4,1019-2016,0.151743


### Mutate EFTSL data to produce primary Broad Field of Education (BFOE), primary BFOE proportion, BFOE entropy and BFOE gini impurity

In [8]:
def build_bfoe():
    '''a function to generate eftsl and bfoe-related features'''

    # aggregate eftsl at BFOE type by provider by year
    df = eftsl.drop(['provider_code', 'ref_year'], axis=1).groupby(['primary_BFOE','code_year']).sum().reset_index()

    # pivot wider and replace NAs with zeros to ensure each provider in each year has a value for each BFOE
    df = df.pivot(index='code_year', columns='primary_BFOE', values='EFTSL').fillna(value=0)

    # rename BFOE categories to be shorter
    df = df.rename(columns = {'01 Natural and Physical Sciences': 'nat_phys_sci',
                             '02 Information Technology': 'info_tech',
                             '03 Engineering and Related Technologies': 'engineering',
                             '04 Architecture and Building': 'arch_build',
                             '05 Agriculture, Environmental and Related Studies': 'agri_env',
                             '06 Health': 'health',
                             '07 Education': 'education',
                             '08 Management and Commerce': 'mge_com',
                             '09 Society and Culture': 'soc_cult',
                             '10 Creative Arts': 'creat_art',
                             '11 Food, Hospitality and Personal Services': 'food_hosp',
                             '12 Mixed Field Programmes': 'mixed',
                             '13 Non-award courses': 'non_award'})

    vars = ['nat_phys_sci', 
            'info_tech', 
            'engineering', 
            'arch_build', 
            'agri_env', 
            'health', 
            'education', 
            'mge_com', 
            'soc_cult', 
            'creat_art', 
            'food_hosp',
            'mixed',
            'non_award']

    # flatten index after pivoting
    df.columns = [''.join(col).strip() for col in df.columns.values]
    df.reset_index(inplace = True)

    # return to long format to implement column-wise calculations
    df = pd.melt(df, id_vars=['code_year'], value_vars=vars, var_name='bfoe', value_name='eftsl')

    # find max bfoe eftsl for each provider
    df['bfoe_max'] = df.groupby(['code_year'])['eftsl'].transform(max)
    df['bfoe_max'].replace(0, np.nan, inplace=True)

    # caclulate total eftsl for ech provider
    df['eftsl_sum'] = df.groupby(['code_year'])['eftsl'].transform(sum)

    # calculate eftsl proportion for each bfoe for each provider for each year
    df['eftsl_prop'] = df['eftsl'] / df['eftsl_sum']
    df['eftsl_prop'].replace(0, np.nan, inplace=True)

    # calculate values to be summed for entropy calculation
    df['pre_entropy'] = -1*df['eftsl_prop']*np.log2(df['eftsl_prop'])

    #calculate entropy
    df['bfoe_entropy'] = df.groupby(['code_year'])['pre_entropy'].transform(sum)

    # calculate values to be summed for gini purity calculation
    df['pre_gini'] = df['eftsl_prop']*(1-df['eftsl_prop'])

    # calculate gini impurity
    df['bfoe_gini_impurity'] = df.groupby(['code_year'])['pre_gini'].transform(sum)

    #filter to only the max bfoe for any given year
    df = df[df['bfoe_max'] == df['eftsl']] 
    df = df[df['bfoe_max'] != 0]

    # drop unnecessary columns and rename as required
    df = df.drop(['eftsl', 'bfoe_max', 'pre_entropy', 'pre_gini'], axis=1).rename(
        columns={'bfoe': 'primary_bfoe', 'eftsl_prop': 'primary_bfoe_prop'})

    # check for multiple code_year references as required
    #import collections
    #print([item for item, count in collections.Counter(df['code_year']).items() if count > 1])

    return df

build_bfoe().head()

,code_year,primary_bfoe,eftsl_sum,primary_bfoe_prop,bfoe_entropy,bfoe_gini_impurity
12,1055-2012,nat_phys_sci,17923.70,0.263659,2.728352,0.822510
13,1055-2013,nat_phys_sci,18345.95,0.331593,2.574494,0.793066
14,1055-2014,nat_phys_sci,18716.07,0.374708,2.423151,0.765983
15,1055-2015,nat_phys_sci,17519.70,0.374720,2.427423,0.767928
16,1055-2016,nat_phys_sci,17633.70,0.368120,2.461615,0.773461


### Load 'TEQSA type' averages for staff data
This data was extracted from TEQSA's 2019 report on higher education statistics. It is included here in the hope that it may assist in missing value imputation where this is required for later analysis.

In [9]:
TEQSA_avg = pd.read_csv('TEQSA_averages.csv')

TEQSA_avg.head()

,TEQSA_type,ref_year,ssr_all_type_avg,ssr_salaried_type_avg,sessional_prop_type_avg
0,university,2013,16.293438,20.590019,0.208673
1,university,2014,16.403840,20.915204,0.215698
2,university,2015,16.512985,21.379006,0.227607
3,university,2016,16.638239,21.690949,0.232941
4,university,2017,16.711462,22.008465,0.240680


### Join all dataframes

In [10]:
def join_data():
    '''a function to join all higher ed data frames''' 
    
    # load datasets
    provider = static()
    performance = load_performance()
    staff = load_staff()
    bfoe = build_bfoe()
    postgrad = build_postgrad()
    international = build_international()
    
    # join datasets
    df = provider.merge(performance, on='provider_code', how = 'outer').merge(
        staff, on='code_year', how = 'outer').merge(
        bfoe, on='code_year', how = 'outer').merge(
        postgrad, on='code_year', how = 'outer').merge(
        international, on='code_year', how = 'outer').merge(TEQSA_avg, on=['TEQSA_type', 'ref_year'], how = 'outer')
    
    # remove providers without a name
    df = df[df.provider_name.notnull()]
    
    # remove ref_year before 2012
    df = df[df['ref_year'] > 2011]
    
    df.ref_year = df.ref_year.astype(int)
    
    # remove providers that no longer existed in the last year of the data collection (2017)
    still_exists = list(df[df['ref_year'] == 2017]['prv'])
    df = df[df.prv.isin(still_exists)]
    
    # remove redundant provider code
    df = df.drop(['provider_code'], axis=1)
    
    df = df[['code_year',
            'prv',
            'provider_name',
            'ref_year',
            'type',
            'TEQSA_type',
            'ownership',
            'profit',
            'affiliation',
            'affil_status',
            'eftsl_sum',
            'primary_bfoe',
            'primary_bfoe_prop',
            'bfoe_entropy',
            'bfoe_gini_impurity',
            'postgrad_prop',
            'int_prop',
            'all_fte',
            'sessional_prop_type_avg',
            'sessional_prop',
            'senior_prop',
            'ssr_all_type_avg',
            'ssr_salaried_type_avg',
            'ssr_all',
            'ssr_salaried',
            'success_dom',
            'success_int',
            'success_all',
            'attrition_dom',
            'attrition_int',
            'attrition_all']]
    
    return df.round(decimals=2)

df = join_data()
df

,code_year,prv,provider_name,ref_year,type,TEQSA_type,ownership,profit,affiliation,affil_status,...,ssr_all_type_avg,ssr_salaried_type_avg,ssr_all,ssr_salaried,success_dom,success_int,success_all,attrition_dom,attrition_int,attrition_all
0,1019-2012,PRV12077,James Cook University,2012,university,university,public,not_for_profit,IRU,current,...,NaN,NaN,13.32,15.67,85.75,83.43,85.43,20.90,15.79,20.20
1,1034-2012,PRV12163,Murdoch University,2012,university,university,public,not_for_profit,IRU,current,...,NaN,NaN,14.23,19.11,80.76,85.90,81.48,17.71,8.36,16.56
2,1055-2012,PRV12169,The University of Western Australia,2012,university,university,public,not_for_profit,Go8,current,...,NaN,NaN,11.00,11.83,90.63,92.87,91.01,8.71,4.96,8.14
3,1058-2012,PRV12062,University of Wollongong,2012,university,university,public,not_for_profit,non_aligned,current,...,NaN,NaN,13.16,16.79,90.45,86.22,89.63,11.67,5.76,10.13
4,2154-2012,PRV12151,Federation University Australia,2012,university,university,public,not_for_profit,RUN,current,...,NaN,NaN,22.41,32.21,88.46,77.44,83.62,19.65,19.65,19.65
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
767,4468-2017,PRV14272,South Metropolitan TAFE,2017,TAFE,TAFE,public,not_for_profit,NUHEP,current,...,13.4,18.37,NaN,NaN,78.79,79.52,79.32,25.71,23.60,24.19
768,7073-2017,PRV12123,Chisholm Institute,2017,TAFE,TAFE,public,not_for_profit,NUHEP,current,...,13.4,18.37,NaN,NaN,85.13,67.90,75.10,21.95,35.29,29.35
769,7075-2017,PRV12049,Technical and Further Education Commission,2017,TAFE,TAFE,public,not_for_profit,NUHEP,current,...,13.4,18.37,NaN,NaN,85.20,77.65,82.81,24.45,39.51,28.81
770,7338-2017,PRV14002,TAFE SA,2017,TAFE,TAFE,public,not_for_profit,NUHEP,current,...,13.4,18.37,NaN,NaN,73.21,73.54,73.30,48.48,16.67,43.59


### Save .csv for further analysis

In [11]:
df.to_csv('higher_ed_data_joined.csv', index=False)